# Global Power Plant

## Advanced Geospatial Visualization with Plotly

In [ ]:
#-----------------------------------------------------------------------------------------
# Name:        Practice Day 9
# Author:      Erick Rico
# Created:     11/08/2025
#-----------------------------------------------------------------------------------------

#### Preparación del Entorno

In [ ]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv("global_power_plant_database.csv")

#### Filtrado y Muestreo

In [3]:
df.shape

(29910, 24)

In [4]:
df.columns

Index(['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data',
       'generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015',
       'generation_gwh_2016', 'generation_gwh_2017',
       'estimated_generation_gwh'],
      dtype='object')

In [12]:
df_sample = df[["country", "latitude", "longitude", "capacity_mw", "primary_fuel"]].sample(5000).copy().reset_index(drop=True)

In [14]:
df_sample.sample(5)

,country,latitude,longitude,capacity_mw,primary_fuel
4295,USA,42.4691,-71.2555,1.6,Solar
4696,KAZ,42.9000,71.3667,1230.0,Coal
1615,USA,32.6825,-116.9888,2.0,Storage
2191,JPN,42.6066,142.5569,200.0,Hydro
524,NOR,63.5260,8.7793,55.0,Wind


#### Creación del Mapa Interactivo

In [ ]:
scat_map = px.scatter_mapbox(df_sample,
                             lat="latitude",
                             lon="longitude",
                             color="primary_fuel",
                             size="capacity_mw",
                             hover_data={"country", "capacity_mw", "primary_fuel"},
                             mapbox_style="carto-positron") # agregado en esta sección porque de otra forma no se muestra el output

scat_map.show()

#### Personalización del Mapa

In [26]:
scat_map = px.scatter_mapbox(df_sample,
                             lat="latitude",
                             lon="longitude",
                             color="primary_fuel",
                             size="capacity_mw",
                             hover_data={"country", "capacity_mw", "primary_fuel"},
                             mapbox_style="carto-positron",
                             size_max=15)

scat_map.update_layout(title="Distribución de Plantas de Energía Globales (Muestra de 5000)",
                       title_x=0.5,
                       mapbox_center={"lat": 30, "lon": 0},
                       mapbox_zoom=0)

scat_map.show()

### Tarea adicional: Mapa Coroplético con Plotly

#### Agregación de Datos

In [31]:
chor_df = df.groupby(by="country", as_index=False).agg({
    "capacity_mw" : pd.Series.sum
}).rename(columns={
    "country": "country_code",
    "capacity_mw": "total_capacity",
})

chor_df.sort_values("total_capacity", ascending=False, inplace=True)
chor_df.sample(5)

,country_code,total_capacity
102,MMR,3937.10
133,SDN,2798.11
81,KEN,2430.20
155,URY,4209.12
25,CAN,143455.20


#### Preparación para el Mapa

In [28]:
import pycountry

def iso3_to_country(code):
    try:
        return pycountry.countries.get(alpha_3=code).name
    except AttributeError:
        return None

In [34]:
chor_df["country_name"] = chor_df["country_code"].apply(iso3_to_country)
chor_df.insert(0, "country_name", chor_df.pop("country_name"))
chor_df

,country_name,country_code,total_capacity
28,China,CHN,1363282.780
156,United States,USA,1193616.450
70,India,IND,288694.111
130,Russian Federation,RUS,227555.550
79,Japan,JPN,211667.750
...,...,...,...
10,Burundi,BDI,49.570
24,Central African Republic,CAF,33.664
47,Western Sahara,ESH,23.400
60,Guinea-Bissau,GNB,18.237


#### Creación del Mapa Coroplético

In [36]:
world_map = px.choropleth(chor_df,
                          locations='country_code',
                          color='total_capacity', 
                          hover_name='country_name', 
                          color_continuous_scale=px.colors.sequential.matter)

world_map.update_layout(coloraxis_showscale=True,
                        title="Capacidad Total de Energía por País")

world_map.show()

#### Comparación

**Conclusiones:**

- Al comparar el mapa coroplético con el scatter mapbox, se observa que el coroplético permite visualizar de manera mucho más clara la capacidad total de generación de energía por país, gracias a la escala de colores: a mayor intensidad, mayor capacidad. Países como Estados Unidos y China destacan notablemente en ambos mapas, pero en el coroplético su liderazgo es aún más evidente.

- En el scatter mapbox, aunque se percibe la alta concentración de plantas en estos países debido al tamaño de los círculos—y al propio tamaño del país en casos específicos—, en regiones como Europa resulta difícil distinguir la capacidad de cada país sin hacer zoom, ya que los puntos se sobreponen (overplotting), incluso usando solo una muestra de 5000 registros de casi 30,000. Esto dificulta identificar qué país sobresale en la región.

- El mapa coroplético, en cambio, facilita identificar rápidamente países destacados como Francia o Alemania dentro de la Unión Europea, o Japón en Asia, donde en el scatter mapbox la información se perdía entre muchos círculos. Además, permite ver claramente que Arabia Saudita lidera en capacidad energética en su zona, algo que en el scatter mapbox no era evidente debido a la superposición de puntos entre países vecinos como Qatar, Bahréin o Emiratos Árabes Unidos.

- En resumen, el mapa coroplético es mucho más efectivo para mostrar comparativamente la capacidad total de energía por país y para obtener una visión global clara y rápida.